In [61]:
# -*- coding: utf8 -*-
from os.path import expanduser
import codecs,yaml
import itertools as it
import ParFuMor as PFM

In [62]:
home = expanduser("~")
annee="17"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
nomTraductions=repertoire+"Phrases-B2-autruches-comp.csv"
nomLexique=repertoire+"%s-K5/"%annee+"stems.yaml"

In [63]:
with codecs.open(nomTraductions,"r",encoding="utf8") as input:
    traductionsIn=input.readlines()
with open(nomLexique, 'rb') as input:
    lexique=yaml.load(input)

In [64]:
def getBottomList(dictLexique):
    listResult=[]
    for cle in dictLexique:
        if isinstance(dictLexique[cle],list):
            listResult.append(dictLexique[cle])
        else:
            listResult.extend(getBottomList(dictLexique[cle]))
    return listResult

def findFormes(forme):
    for verbe in verbes:
        if forme in verbe:
            vsg=verbe[1::2]
            print vsg
            vpl=verbe[2::2]
            print vpl
            if forme in vsg:
                nombre="sg"
            elif forme in vpl:
                nombre="pl"
            else:
                print "problème avec la personne du verbe"
    return (nombre,vsg,vpl)

In [65]:
verbes=getBottomList(lexique["VER"])
findFormes("tombait")

['tombe', 'tombait']
['tombent', 'tombaient']


('sg', ['tombe', 'tombait'], ['tombent', 'tombaient'])

In [58]:
def getAlternatives(libelle,syntagme):
    result=set()
    alternatives=raw_input(libelle+syntagme+"\n").decode("utf8")
    for element in alternatives.split(","):
        alternative=element.strip()
        if "MS" in alternative:
            for det in ["le","ce","un"]:
                result.add(alternative.replace("MS",det))
        elif "MV" in alternative:
            for det in ["l'","cet","un"]:
                result.add(alternative.replace("MV",det).replace("' ","'"))
        elif "FS" in alternative:
            for det in ["la","cette","une"]:
                result.add(alternative.replace("FS",det))
        elif "FV" in alternative:
            for det in ["l'","cette","une"]:
                result.add(alternative.replace("FV",det).replace("' ","'"))
        elif "PL" in alternative:
            for det in ["les","ces","des","les deux","ces deux","deux"]:
                result.add(alternative.replace("PL",det))
        elif "AUX" in alternative:
            for det in ["aux",u"à ces",u"à des","aux deux",u"à ces deux",u"à deux"]:
                result.add(alternative.replace("AUX",det))
        elif "AU" in alternative:
            for det in ["au",u"à ce",u"à un"]:
                result.add(alternative.replace("AU",det))
        elif "DU" in alternative:
            for det in ["du","de ce","d'un"]:
                result.add(alternative.replace("DU",det))
        elif "DES" in alternative:
            for det in ["des","de ces","de","des deux","de ces deux","de deux"]:
                result.add(alternative.replace("DES",det))
        else:
            result.add(alternative)
    return result         

In [59]:
phrasesOut=set()
alternatives={'':[""]}
verbes=getBottomList(lexique["VER"])
for ligne in traductionsIn[:5]:
    syntagmes=ligne.strip().split("\t")
    gnSG={}
    gnPL={}
    vSG=[]
    vPL=[]
    divers={}
    print " ".join(syntagmes)
    (nombreSujet,vSG,vPL)=findFormes(syntagmes[1])
    for nSyntagme,syntagme in enumerate(syntagmes):
        if syntagme:
            if syntagme in alternatives:
                print syntagme, ", ".join(list(alternatives[syntagme]))
            else:
                alternatives[syntagme]=set()
            if nSyntagme in [0,2]:
                if not syntagme in gnSG: gnSG[syntagme]=set()
                if not syntagme in gnPL: gnPL[syntagme]=set()
                gnSG[syntagme]|=getAlternatives("alternatives SG pour ",syntagme.encode("utf8"))
                gnPL[syntagme]|=getAlternatives("alternatives PL pour ",syntagme.encode("utf8"))
                if nombreSujet=="sg": gnSG[syntagme]|=set([syntagme])
                else: gnPL[syntagme]|=set([syntagme])
                alternatives[syntagme]|=gnSG[syntagme]|gnPL[syntagme]
            elif not nSyntagme in [1]:
                alternatives[syntagme]|=getAlternatives("alternatives pour ",syntagme.encode("utf8"))
                alternatives[syntagme]|=set([syntagme])
    altSyntagmes=[list(alternatives[k]) for k in syntagmes[2:]]
    sujetSG=[gnSG[syntagmes[0]],vSG]+altSyntagmes
    sujetPL=[gnPL[syntagmes[0]],vPL]+altSyntagmes
    for phrase in it.product(*sujetSG):
        phrase=list(phrase)
        if not phrase[-1].endswith("."): phrase[-1]+="."
        phrasesOut.add("\t".join(phrase))
    for phrase in it.product(*sujetPL):
        phrase=list(phrase)
        if not phrase[-1].endswith("."): phrase[-1]+="."
        phrasesOut.add("\t".join(phrase))

les chasseurs offrent deux coyotes à l'infirmière.
['offre', 'offrait']
['offrent', 'offraient']
alternatives SG pour les chasseurs
MS chasseur
alternatives PL pour les chasseurs
PL chasseurs
alternatives SG pour deux coyotes
MS coyote
alternatives PL pour deux coyotes
PL coyotes
alternatives pour à l'infirmière.
à FV infirmière,AUX infirmières
l'infirmière offre un coyote aux chasseurs.
['offre', 'offrait']
['offrent', 'offraient']
alternatives SG pour l'infirmière
FV infirmière
alternatives PL pour l'infirmière
PL infirmières
alternatives SG pour un coyote
MS coyote
alternatives PL pour un coyote
PL coyotes
alternatives pour aux chasseurs.
AUX chasseurs,AU chasseur
les deux fruits arrivaient   dans la cuisine de Nicole.
['arrive', 'arrivait']
['arrivent', 'arrivaient']
alternatives SG pour les deux fruits
MS fruit
alternatives PL pour les deux fruits
PL fruits
alternatives pour dans la cuisine de Nicole.
dans FS cuisine de Nicole,dans PL cuisines de Nicole
Nicole arrive   dans la cui

In [60]:
for nPhrase,phrase in enumerate(sorted(phrasesOut)):
    print nPhrase, phrase

0 Nicole	arrivait			dans cette cuisine des fruits.
1 Nicole	arrivait			dans la cuisine de ces deux fruits.
2 Nicole	arrivait			dans la cuisine de ces fruits.
3 Nicole	arrivait			dans la cuisine de deux fruits.
4 Nicole	arrivait			dans la cuisine de fruits.
5 Nicole	arrivait			dans la cuisine des deux fruits.
6 Nicole	arrivait			dans la cuisine des fruits.
7 Nicole	arrivait			dans une cuisine des fruits.
8 Nicole	arrive			dans cette cuisine des fruits.
9 Nicole	arrive			dans la cuisine de ces deux fruits.
10 Nicole	arrive			dans la cuisine de ces fruits.
11 Nicole	arrive			dans la cuisine de deux fruits.
12 Nicole	arrive			dans la cuisine de fruits.
13 Nicole	arrive			dans la cuisine des deux fruits.
14 Nicole	arrive			dans la cuisine des fruits.
15 Nicole	arrive			dans une cuisine des fruits.
16 ce chasseur	offrait	ce coyote	aux deux infirmières.
17 ce chasseur	offrait	ce coyote	aux infirmières.
18 ce chasseur	offrait	ce coyote	à ces deux infirmières.
19 ce chasseur	offrait	ce coyote	à

966 ces infirmières	offraient	les deux coyotes	à ces deux chasseurs.
967 ces infirmières	offraient	les deux coyotes	à des chasseurs.
968 ces infirmières	offraient	les deux coyotes	à deux chasseurs.
969 ces infirmières	offraient	les deux coyotes	à un chasseur.
970 ces infirmières	offraient	un coyote	au chasseur.
971 ces infirmières	offraient	un coyote	aux chasseurs.
972 ces infirmières	offraient	un coyote	aux deux chasseurs.
973 ces infirmières	offraient	un coyote	à ce chasseur.
974 ces infirmières	offraient	un coyote	à ces chasseurs.
975 ces infirmières	offraient	un coyote	à ces deux chasseurs.
976 ces infirmières	offraient	un coyote	à des chasseurs.
977 ces infirmières	offraient	un coyote	à deux chasseurs.
978 ces infirmières	offraient	un coyote	à un chasseur.
979 ces infirmières	offrent	ce coyote	au chasseur.
980 ces infirmières	offrent	ce coyote	aux chasseurs.
981 ces infirmières	offrent	ce coyote	aux deux chasseurs.
982 ces infirmières	offrent	ce coyote	à ce chasseur.
983 ces infir

2525 deux petites tables jaunes	tombaient			devant les trois chats de Nabil	dans les deux cuisines.
2526 deux petites tables jaunes	tombaient			devant les trois chats de Nabil	dans une cuisine.
2527 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans ces cuisines.
2528 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans ces deux cuisines.
2529 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans cette cuisine.
2530 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans des cuisines.
2531 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans deux cuisines.
2532 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans la cuisine.
2533 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans les cuisines.
2534 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans les deux cuisines.
2535 deux petites tables jaunes	tombaient			devant un chat de Nabil	dans une cuisine.
2536 deux peti

4025 les infirmières	offrent	un coyote	à des chasseurs.
4026 les infirmières	offrent	un coyote	à deux chasseurs.
4027 les infirmières	offrent	un coyote	à un chasseur.
4028 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans ces cuisines.
4029 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans ces deux cuisines.
4030 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans cette cuisine.
4031 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans des cuisines.
4032 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans deux cuisines.
4033 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans la cuisine.
4034 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans les cuisines.
4035 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans les deux cuisines.
4036 les petites tables jaunes	tombaient			devant ce chat de Nabil	dans une cuisine.
4037 les petites tables jaunes	tombaient			devant 

In [52]:
fonctions=["Sujet","Verbe","COD","COI","COMP","AJOUT"]
for traduction in traductionsIn[:]:
    for fonction in fonctions:
        traduction=traduction.replace("\t",":"+fonction+u" | ",1)
    print traduction.strip()

les chasseurs:Sujet | offrent:Verbe | deux coyotes:COD | à l'infirmière.:COI | :COMP |
l'infirmière:Sujet | offre:Verbe | un coyote:COD | aux chasseurs.:COI | :COMP |
les deux fruits:Sujet | arrivaient:Verbe | :COD | :COI | dans la cuisine de Nicole.:COMP |
Nicole:Sujet | arrive:Verbe | :COD | :COI | dans la cuisine des fruits.
les deux petites tables jaunes:Sujet | tombent:Verbe | :COD | :COI | devant les trois chats de Nabil:COMP | dans la cuisine.
les trois chats de Nabil:Sujet | tombent:Verbe | :COD | :COI | devant les petites tables jaunes:COMP | dans la cuisine.
ce grand chasseur de coyotes:Sujet | donne:Verbe | une maison:COD | aux infirmières du village.:COI | :COMP |
les infirmières du village:Sujet | donnent:Verbe | une maison:COD | à ce grand chasseur de coyotes.:COI | :COMP |
les trois villages de la plaine:Sujet | chassent:Verbe | les chasseurs d'autruches :COD | :COI | avec des balais.:COMP |
les chasseurs d'autruches:Sujet | chassent:Verbe | les trois villages de la plai